In [3]:
from IPython import display

import nltk

import numpy as np
import pandas as pd

import hazm
import requests
import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from sentence_transformers import CrossEncoder

document = 'های بازرگانی از قبیل صادرات و واردات، خرید و فروش و توزیع کالاهای مجاز بازرگانی، بازاریابی، اخذ تسهیلات از بانکهای دولتی و خصوصی و یا شرکتهای سرمایه گذاری ایرانی و خارجی صرفا جهت تحقق اهداف شرکت، شرکت در مناقصات و مزایدات دولتی و خصوصی، اخذ و اعطای نمایندگی از شرکتهای دولتی و خصوصی و خارجی و داخلی، مشارکت با شرکتها و موسسات و سازمانها اعم از دولتی و خصوصی در کلیه زمینه های اقتصادی و بازرگانی در داخل و خارج کشور.درصورت لزوم پس از اخذ مجوزهای لازم از مراجع ذیربط ـ ثبت موضوع به تنهایی مجوز فعالیت نمی باشد '

paragraphs = []
for paragraph in document.replace("\r\n", "\n").split("\n\n"):
    if len(paragraph.strip()) > 0:
        sentences = hazm.sent_tokenize(paragraph.strip())
        paragraphs.append(sentences)

window_size = 3
passages = []
for paragraph in paragraphs:
    for start_idx in range(0, len(paragraph), window_size):
        end_idx = min(start_idx + window_size, len(paragraph))
        passages.append(" ".join(paragraph[start_idx:end_idx]))


print(f"Paragraphs: {len(paragraphs)}")
print(f"Sentences: {sum([len(p) for p in paragraphs])}")
print(f"Passages: {len(passages)}")

Paragraphs: 1
Sentences: 1
Passages: 1


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
txt = "های بازرگانی از قبیل صادرات و واردات، خرید و فروش و توزیع کالاهای مجاز بازرگانی، بازاریابی، اخذ تسهیلات از بانکهای دولتی و خصوصی و یا شرکتهای سرمایه گذاری ایرانی و خارجی صرفا جهت تحقق اهداف شرکت، شرکت در مناقصات و مزایدات دولتی و خصوصی، اخذ و اعطای نمایندگی از شرکتهای دولتی و خصوصی و خارجی و داخلی، مشارکت با شرکتها و موسسات و سازمانها اعم از دولتی و خصوصی در کلیه زمینه های اقتصادی و بازرگانی در داخل و خارج کشور.درصورت لزوم پس از اخذ مجوزهای لازم از مراجع ذیربط ـ ثبت موضوع به تنهایی مجوز فعالیت نمی باشد"
txt_enc = tokenizer(txt)

tokenizer.convert_ids_to_tokens(txt_enc.input_ids) # returns ['[CLS]', 'the', 'hi', '##ggs', 'field', 'gives', 'mass', 'to', 'sub', '##ato', '##mic', 'particles', '[SEP]']
tokenizer.decode(txt_enc.input_ids) # returns "[CLS] the higgs field gives mass to subatomic particles [SEP]"

'[CLS] های بازرگانی از قبیل صادرات و واردات ، خرید و فروش و توزیع کالاهای مجاز بازرگانی ، بازاریابی ، اخذ تسهیلات از بانکهای دولتی و خصوصی و یا شرکتهای سرمایه گذاری ایرانی و خارجی صرفا جهت تحقق اهداف شرکت ، شرکت در مناقصات و مزایدات دولتی و خصوصی ، اخذ و اعطای نمایندگی از شرکتهای دولتی و خصوصی و خارجی و داخلی ، مشارکت با شرکتها و موسسات و سازمانها اعم از دولتی و خصوصی در کلیه زمینه های اقتصادی و بازرگانی در داخل و خارج کشور. درصورت لزوم پس از اخذ مجوزهای لازم از مراجع ذیربط ـ ثبت موضوع به تنهایی مجوز فعالیت نمی باشد [SEP]'

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2")
txt = "های بازرگانی از قبیل صادرات و واردات، خرید و فروش و توزیع کالاهای مجاز بازرگانی، بازاریابی، اخذ تسهیلات از بانکهای دولتی و خصوصی و یا شرکتهای سرمایه گذاری ایرانی و خارجی صرفا جهت تحقق اهداف شرکت، شرکت در مناقصات و مزایدات دولتی و خصوصی، اخذ و اعطای نمایندگی از شرکتهای دولتی و خصوصی و خارجی و داخلی، مشارکت با شرکتها و موسسات و سازمانها اعم از دولتی و خصوصی در کلیه زمینه های اقتصادی و بازرگانی در داخل و خارج کشور.درصورت لزوم پس از اخذ مجوزهای لازم از مراجع ذیربط ـ ثبت موضوع به تنهایی مجوز فعالیت نمی باشد"
txt_enc = tokenizer(txt)

tokenizer.convert_ids_to_tokens(txt_enc.input_ids) # returns ['[CLS]', 'the', 'hi', '##ggs', 'field', 'gives', 'mass', 'to', 'sub', '##ato', '##mic', 'particles', '[SEP]']
tokenizer.decode(txt_enc.input_ids) # returns "[CLS] the higgs field gives mass to subatomic particles [SEP]"

'[CLS] های بازرگانی از قبیل صادرات و واردات ، خرید و فروش و توزیع کالاهای مجاز بازرگانی ، بازاریابی ، اخذ تسهیلات از بانکهای دولتی و خصوصی و یا شرکتهای سرمایه گذاری ایرانی و خارجی صرفا جهت تحقق اهداف شرکت ، شرکت در مناقصات و مزایدات دولتی و خصوصی ، اخذ و اعطای نمایندگی از شرکتهای دولتی و خصوصی و خارجی و داخلی ، مشارکت با شرکتها و موسسات و سازمانها اعم از دولتی و خصوصی در کلیه زمینه های اقتصادی و بازرگانی در داخل و خارج کشور. درصورت لزوم پس از اخذ مجوزهای لازم از مراجع ذیربط ـ ثبت موضوع به تنهایی مجوز فعالیت نمی باشد [SEP]'